In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tqdm
from scipy.spatial.transform import Rotation as R

In [ ]:
DATASET_PATH = "dataset.h5"
INTRINSICS = {"focalLength": 1.0, "width": 384.0, "height": 256.0, "zRange": [0.01, 2.0]}
VIEW_IDX = 3

In [ ]:
with h5py.File(DATASET_PATH, "r") as f:
    dp_key = next(iter(f.keys()))
    for k, v in f[dp_key].items():
        print(f"{k:<12} {v.shape}")

In [ ]:
def create_distance_map(target_poses, cam_pose, depth):
    h, w = int(INTRINSICS["height"]), int(INTRINSICS["width"])
    f = INTRINSICS["focalLength"]
    Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing="ij")

    # Unproject to camera space
    x = (X - w / 2) / (w / 2) * depth / f
    y = (Y - h / 2) / (h / 2) * depth / f
    pts_cam = np.stack([x, y, depth], axis=-1)

    # Transform to world space
    rot = R.from_quat(np.roll(cam_pose[3:], -1)).as_matrix()
    pts_world = (rot @ pts_cam.reshape(-1, 3).T).T.reshape(h, w, 3) + cam_pose[:3]

    # Compute distances
    dists = [np.linalg.norm(pts_world - pose[:3], axis=-1) for pose in target_poses]
    return np.stack(dists).astype(np.float32)

In [ ]:
with h5py.File(DATASET_PATH, "r") as f:
    depths, dists = [], []

    for key in tqdm.tqdm(f.keys()):
        dp = f[key]
        cam_pose = dp["cam_poses"][VIEW_IDX][()]
        depth = dp["depths"][VIEW_IDX][()]
        feasibles = dp["feasibles"][()].astype(bool)

        depths.append(depth)
        dists.append(create_distance_map(dp["target_poses"][()], cam_pose, depth).flatten())

depths = np.concatenate(depths)
dists = np.concatenate(dists)

In [ ]:
depth_min = depths.min()
depth_max = depths.max()
dist_max = dists.max()

print(f"depth:    [{depth_min:.4f}, {depth_max:.4f}], mean={depths.mean():.4f}")
print(f"distance: [{dists.min():.4f}, {dist_max:.4f}], mean={dists.mean():.4f}")

In [ ]:
with h5py.File(DATASET_PATH, "r") as f_in, h5py.File("dataset-distance.h5", "w") as f_out:
    f_out.attrs.update(f_in.attrs)
    f_out.attrs["depth_min"] = depth_min
    f_out.attrs["depth_max"] = depth_max
    f_out.attrs["dist_max"] = dist_max

    for key in tqdm.tqdm(f_in.keys()):
        dp = f_in[key]

        obj_ids = dp["obj_ids"][()]
        cam_pose = dp["cam_poses"][VIEW_IDX][()]
        seg = dp["seg_ids"][VIEW_IDX]
        poses = dp["poses"][()]
        target_poses = dp["target_poses"][()]
        feasibles = dp["feasibles"][()]

        depth_raw = dp["depths"][VIEW_IDX][()]
        depth = (depth_raw - depth_min) / (depth_max - depth_min)

        dist_maps = create_distance_map(target_poses, cam_pose, depth_raw)
        dist_maps = np.clip(dist_maps / dist_max, 0.0, 1.0)

        for i in range(len(obj_ids)):
            grp = f_out.create_group(f"{key}_obj_{i}")
            grp.create_dataset("depth", data=depth)
            grp.create_dataset("mask", data=(seg == obj_ids[i]).astype(np.float32))
            grp.create_dataset("dist_map", data=dist_maps[i])
            grp.create_dataset("quat", data=poses[i][3:])
            grp.create_dataset("target_pos", data=target_poses[i])
            grp.create_dataset("feasibles", data=feasibles[i])

In [ ]:
def visualize_sample(depth, mask, dist_map, threshold=0.1):
    img = np.stack([depth, depth, depth], axis=-1)
    img[mask > 0.5, 1] += 0.5  # Green: object
    print(dist_map.min(), dist_map.max())
    img[:, :, 0] += ((1 - dist_map / threshold).clip(0, 1) ** 2) * 0.7
    img = img.clip(0, 1)

    plt.imshow(img)
    plt.axis("off")
    plt.tight_layout()

In [ ]:
with h5py.File("dataset-distance.h5", "r") as f:
    keys = list(f.keys())
    dp = f[keys[7]]
    visualize_sample(dp["depth"][()], dp["mask"][()], dp["dist_map"][()])
    plt.show()